In [56]:
# !pip install xgboost

In [69]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
import hydroeval as he

In [58]:
pred_basel = pd.read_csv("basel.csv",index_col=0)
pred_basel

,datetime,et,p,t,obs
0,1981-01-01,0.000405,0.003536,-0.250816,670.000
1,1981-01-02,0.000472,0.003379,2.172385,647.000
2,1981-01-03,0.000576,0.017352,5.165919,727.000
3,1981-01-04,0.000443,0.022734,1.019687,1363.000
4,1981-01-05,0.000364,0.005195,-2.055391,1202.000
...,...,...,...,...,...
7300,2000-12-27,0.000492,0.001242,2.119268,662.035
7301,2000-12-28,0.000413,0.000561,-0.594725,656.253
7302,2000-12-29,0.000348,0.002027,-3.251007,652.150
7303,2000-12-30,0.000306,0.000741,-5.414832,641.549


In [59]:
q_basel = pd.read_csv("q_basel.csv",index_col=0)
q_basel

,obs,pcr,res
datetime,,,
1981-01-01,670.000,605.06915,64.93085
1981-01-02,647.000,599.35767,47.64233
1981-01-03,727.000,874.38354,-147.38354
1981-01-04,1363.000,998.86804,364.13196
1981-01-05,1202.000,968.06270,233.93730
...,...,...,...
2000-12-27,662.035,1055.62610,-393.59110
2000-12-28,656.253,1027.90730,-371.65430
2000-12-29,652.150,860.76306,-208.61306


In [61]:
# predictors and output var
len_ = int(0.80 * pred_basel.shape[0])

df_train = pred_basel[:len_]
df_test = pred_basel[len_:]


X_train = df_train[["t","et","p"]]
y_train = df_train.obs

X_test = df_test[["t","et","p"]]
y_test = df_test.obs
X_test

,t,et,p
5844,-8.276995,0.000366,0.000000
5845,-4.375016,0.000462,0.006172
5846,-0.382900,0.000603,0.006253
5847,-0.382237,0.000600,0.002056
5848,-4.089456,0.000472,0.000000
...,...,...,...
7300,2.119268,0.000492,0.001242
7301,-0.594725,0.000413,0.000561
7302,-3.251007,0.000348,0.002027
7303,-5.414832,0.000306,0.000741


### Normalise data

In [ ]:
##TO DO##

### Model

In [62]:
# create a xgboost regression model
model = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)


In [63]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=7, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: -328.052 (10.894)


In [64]:
# define model
model = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
# fit model
model.fit(X_train, y_train)
# make a prediction
yhat = model.predict(X_test)

### evaluation

In [70]:
streamflow_pcr = q_basel.pcr[len_:]

#evaluate pcr model using nse and kge
nse_pcr = he.evaluator(he.nse, streamflow_pcr,y_test)
kge_pcr, r, alpha, beta = he.evaluator(he.kge, streamflow_pcr, y_test)
print("The nse and kge of the PCR model are {:.2} and {:.2}.".format(nse_pcr[0], kge_pcr[0]))

#evaluate the prediction using nse and kge
nse = he.evaluator(he.nse, yhat, y_test)

kge, r, alpha, beta = he.evaluator(he.kge, yhat, y_test)
print("The nse and kge of the xgboost model are {:.2} and {:.2}.".format(nse[0], kge[0]))

The nse and kge of the PCR model are 0.27 and 0.66.
The nse and kge of the linear model are 0.039 and 0.21.
